This Code will generate a Dashboard (via Dash) in the browser to visualize our analyses regarding the influence from natural disasters on stock markets. On the one side you will see the map with the earthquakes and the organisations depending on what filter you use. On the other side you will see our analysis to the selected industry: Market Recovery Pattern, Days to Recover and Impact from Earthquake on Stock Price. Right now the charts refer only to the japanese market. 

In [7]:
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import plotly.graph_objects as go  
import pandas as pd
import webbrowser
from threading import Timer

Initializing the App and import the Data 

In [ ]:

# Initialize the Dash app
app = dash.Dash(__name__)

# Load Data
disaster_stock_data = pd.read_pickle("/Users/jan/Techlabs_WS_Team3/02_code/0_preprocess/disaster_stock_data.pkl")
nikkei_companies = pd.read_pickle("/Users/jan/Techlabs_WS_Team3/02_code/0_preprocess/nikkei_companies.pkl")
industry_impact_on_prices = pd.read_pickle("/Users/jan/Techlabs_WS_Team3/01_data/02_pre/N225_Japan/industry_impact_on_prices.pkl")
agg_results = pd.read_pickle("/Users/jan/Techlabs_WS_Team3/01_data/02_pre/N225_Japan/agg_results.pkl")
recovery_analysis = pd.read_pickle("/Users/jan/Techlabs_WS_Team3/01_data/02_pre/N225_Japan/recovery_analysis.pkl")

# Standardize Industry Names
nikkei_companies['Industry'] = nikkei_companies['Industry'].str.strip().str.lower()
industry_impact_on_prices['Industry'] = industry_impact_on_prices['Industry'].str.strip().str.lower()
agg_results['Industry'] = agg_results['Industry'].str.strip().str.lower()
recovery_analysis['Industry'] = recovery_analysis['Industry'].str.strip().str.lower()

#prepare Data 
disaster_stock_data['state'] = disaster_stock_data['state'].fillna("").astype(str)

Layout of the App 

In [9]:


# Layout of the app
app.layout = html.Div(
    style={"backgroundColor": "white", "color": "black", "padding": "10px", "minHeight": "100vh"},
    children=[
        html.H2(children='Stock Market Analysis',
                style={'textAlign': 'left', "color": "#8ABF8B"}),

        # Create filter as dropwdowns 
        html.Div(
            style={"display": "flex", "justifyContent": "left", "gap": "20px", "marginBottom": "10px"},
            children=[
                html.Div([
                    html.Label("Year", style={"fontSize": "15px", "color": "black"}),
                    dcc.Dropdown(
                        id='dropdown-selection',
                        options=[{'label': date, 'value': date} for date in disaster_stock_data.year.unique()],
                        value=disaster_stock_data.year.unique()[0],
                        clearable=False,
                        style={"backgroundColor": "white", "color": "black", "width": "250px"},
                        placeholder="Wähle ein Datum",
                    )
                ], style={"display": "flex", "flexDirection": "column"}),

                html.Div([
                    html.Label("State", style={"fontSize": "15px", "color": "black"}),
                    dcc.Dropdown(
                        id='state-selection',
                        options=[{'label': state, 'value': state} for state in disaster_stock_data.state.unique()],
                        value=disaster_stock_data.state.unique()[0],
                        clearable=False,
                        style={"backgroundColor": "white", "color": "black", "width": "250px"},
                        placeholder="Wähle einen State",
                    )
                ], style={"display": "flex", "flexDirection": "column"}),

                html.Div([
                    html.Label("Industry", style={"fontSize": "15px", "color": "black"}),
                    dcc.Dropdown(
                        id='industry-selection',
                        options=[{'label': industry, 'value': industry} for industry in nikkei_companies.Industry.unique()],
                        value=nikkei_companies.Industry.unique()[0],
                        clearable=False,
                        style={"backgroundColor": "white", "color": "black", "width": "250px"},
                        placeholder="Wähle eine Industry",
                    )
                ], style={"display": "flex", "flexDirection": "column"}),
            ],
        ),

        # Layout map and right side of the app
        html.Div(
            id="main-container",
            style={"display": "flex", "gap": "20px", "height": "80vh"},  
            children=[
                html.Div(  
                    id="map-container",
                    style={"flex": "0.5"},
                    children=[dcc.Graph(id='graph-content', style={"height": "100%"})],
                ),
                html.Div(  
                    id="right-panel",
                    style={
                        "flex": "0.5",
                        "backgroundColor": "#f8f9fa",
                        "padding": "20px",
                        "borderRadius": "10px",
                        "display": "flex",
                        "flexDirection": "column",
                        "gap": "20px",
                    },
                    children=[
                        # Market Recovery Chart (top)
                        html.Div(
                            style={"flex": "1"},
                            children=[
                                dcc.Graph(
                                    id='market-recovery-chart',
                                    style={"height": "300px"}
                                ),
                            ],
                        ),

                        # Container for Chart 1 and Chart 2 (side by side)
                        html.Div(
                            style={
                                "display": "flex",
                                "flexDirection": "row",
                                "gap": "20px",
                                "flex": "1",
                            },
                            children=[
                                # Chart 1 (left) - Industry-wise Analysis
                                html.Div(
                                    style={"flex": "1"},
                                    children=[
                                        dcc.Graph(
                                            id='chart-1',
                                            style={"flex": "1",
                                                   "backgroundColor": "white",
                                                   "padding": "20px",
                                                   "borderRadius": "10px",
                                                   "boxShadow": "0px 0px 10px rgba(0, 0, 0, 0.1)"}
                                        ),
                                    ],
                                ),

                                # Chart 2 (right) - Display a number in a white box
                                html.Div(
                                    style={
                                        "flex": "1",
                                        "backgroundColor": "white",
                                        "padding": "20px",
                                        "borderRadius": "10px",
                                        "boxShadow": "0px 0px 10px rgba(0, 0, 0, 0.1)",
                                        "display": "flex",
                                        "flexDirection": "column",
                                        "alignItems": "center",
                                        "justifyContent": "center",
                                    },
                                    children=[
                                        html.H3("Days to Recover",
                                                style={"fontSize": "18px"}),
                                        html.Div(
                                            id='chart-2',
                                            style={
                                                "fontSize": "24px",
                                                "fontWeight": "bold",
                                                "textAlign": "center",
                                                "marginTop": "10px"
                                            }
                                        ),
                                    ],
                                ),
                            ],
                        ),
                    ],
                ),
            ],
        ),
    ]
)





Callbacks

In [10]:
# Update Map depending on the filters you choose
@app.callback(
    Output('graph-content', 'figure'),
    [Input('dropdown-selection', 'value'),
     Input('state-selection', 'value'),
     Input('industry-selection', 'value')]
)
def update_map(selected_date, selected_state, selected_industry):
    filtered_data = disaster_stock_data[
        (disaster_stock_data['year'] == selected_date) &
        (disaster_stock_data['state'] == selected_state)
    ]

    filtered_companies = nikkei_companies[nikkei_companies['Industry'] == selected_industry]

    if filtered_data.empty and filtered_companies.empty:
        return go.Figure()

    # 🌍 Density Map mit CARTO-Style
    fig = px.density_mapbox(
        filtered_data,
        lat="latitude",
        lon="longitude",
        z="magnitudo",
        hover_name="place",
        radius=10,  # Dichte-Radius für bessere Sichtbarkeit
        zoom=5,
        mapbox_style="carto-positron",  # Setzt CARTO als Kartenstil
    )

    # Create marker for companies on the map 
    if not filtered_companies.empty:
        fig.add_trace(go.Scattermapbox(
            lat=filtered_companies["latitude"].tolist(),
            lon=filtered_companies["longitude"].tolist(),
            mode="markers",
            marker=dict(size=15, color="cyan"),
            hovertext=filtered_companies["Company_name"],
            hoverinfo="text",
        ))

    # Layout
    fig.update_layout(
        margin=dict(l=10, r=10, t=20, b=10, pad=5),
        plot_bgcolor="white",
        paper_bgcolor="white",
        clickmode="event+select",
        hovermode="closest",
        coloraxis_showscale=False,
        showlegend=False,
        mapbox=dict(
            accesstoken="pk.eyJ1IjoidGVjaGxhYnMzIiwiYSI6ImNtNzBib2xyczAwZHoycnBiM2hxZ24zcngifQ.AaSHNEfc-cnR4uDdEO4gsw",
            bearing=10,
            center=dict(
                lat=filtered_data.latitude.mean() if not filtered_data.empty else 50,
                lon=filtered_data.longitude.mean() if not filtered_data.empty else 10
            ),
            pitch=5,
            zoom=5,
        ),
    )

    return fig

# Callback for Market Recovery Chart
@app.callback(
    Output('market-recovery-chart', 'figure'),
    Input('industry-selection', 'value')
)
def update_market_recovery_chart(selected_industry):
  
    selected_industry = selected_industry.strip().lower()

    filtered_data = agg_results[agg_results['Industry'] == selected_industry]

    # Create the Plotly line plot
    fig = px.line(
        filtered_data,
        x='days_from_event',
        y='mean_change',
        title=("Market Recovery Pattern"),
        labels={'days_from_event': 'Days Relative to Earthquake', 'mean_change': 'Normalized Price Change (%)'}
    ).update_layout(title_x=0.5)
    fig.add_vline(x=0, line_dash="dash", line_color="red", annotation_text="Earthquake Day")
    fig.add_hline(y=0, line_dash="solid", line_color="black")
    return fig

# Callback for Chart 1 (Industry-wise Analysis)
@app.callback(
    Output('chart-1', 'figure'),
    Input('industry-selection', 'value')
)
def update_chart_1(selected_industry):
   
    selected_industry = selected_industry.strip().lower()

   
    filtered_data = industry_impact_on_prices[industry_impact_on_prices['Industry'] == selected_industry]

    # Create the Plotly bar plot
    fig = px.bar(
        filtered_data,
        x='Industry',
        y='price_change',
        color='earthquake',
        barmode='group',
        title='Impact of Earthquakes on <br>Stock Prices by Industry',
        labels={'price_change': 'Percentage Change in Stock Prices'}
    )
    return fig

# Callback for Chart 2 (display a number)
@app.callback(
    Output('chart-2', 'children'),
    Input('industry-selection', 'value')
)
def update_chart_2(selected_industry):
    
    selected_industry = selected_industry.strip().lower()

    filtered_data = recovery_analysis[recovery_analysis['Industry'] == selected_industry]

    days_to_recover = filtered_data['days_to_recover'].values[0] if not filtered_data.empty else "N/A"

    return f"{days_to_recover} days"

Run and open the map in the Browser

In [ ]:
# Function to open the browser
def open_browser():
    webbrowser.open_new("http://127.0.0.1:8050/") 

# Run the app
if __name__ == '__main__':
    Timer(1, open_browser).start()  
    app.run_server(debug=True)  

/var/folders/xm/s7wz86kd7szcj2pdp0nfk8mh0000gn/T/ipykernel_847/2727064522.py:20: DeprecationWarning:

*density_mapbox* is deprecated! Use *density_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

/var/folders/xm/s7wz86kd7szcj2pdp0nfk8mh0000gn/T/ipykernel_847/2727064522.py:33: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

/var/folders/xm/s7wz86kd7szcj2pdp0nfk8mh0000gn/T/ipykernel_847/2727064522.py:20: DeprecationWarning:

*density_mapbox* is deprecated! Use *density_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

/var/folders/xm/s7wz86kd7szcj2pdp0nfk8mh0000gn/T/ipykernel_847/2727064522.py:33: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

